# בניית סוכני AI עם זיכרון מתמשך באמצעות Cognee

מחברת זו מדגימה כיצד לבנות סוכני AI חכמים עם יכולות זיכרון מתקדמות באמצעות [**cognee**](https://www.cognee.ai/) - זיכרון AI בקוד פתוח שמשלב גרפים של ידע, חיפוש סמנטי וניהול סשנים ליצירת מערכות AI מודעות להקשר.

## 🎯 מטרות למידה

בסיום המדריך הזה, תבינו כיצד:
- **לבנות גרפי ידע המבוססים על Embeddings**: להפוך טקסט לא מובנה לידע מובנה שניתן לשאול עליו שאלות
- **ליישם זיכרון סשן**: ליצור שיחות מרובות שלבים עם שמירה אוטומטית של ההקשר
- **לשמר שיחות**: לשמור אינטראקציות חשובות בזיכרון לטווח ארוך לשימוש עתידי
- **לשאול שאלות בשפה טבעית**: לגשת ולהשתמש בהקשר היסטורי בשיחות חדשות
- **להמחיש את הזיכרון**: לחקור את הקשרים בגרף הידע של הסוכן שלכם


## 🏗️ מה תבנו

במדריך זה ניצור **עוזר קוד** עם זיכרון מתמשך ש:

### 1. **בניית בסיס ידע**
   - קולט מידע על פרופיל המפתח וההתמחות שלו
   - מעבד עקרונות ותובנות של תכנות ב-Python
   - שומר שיחות היסטוריות בין מפתחים לעוזרים מבוססי AI

### 2. **שיחות מודעות להקשר**
   - שומר על הקשר בין שאלות מרובות באותה סשן
   - מאחסן אוטומטית כל זוג שאלה/תשובה לצורך שליפה יעילה
   - מספק תגובות עקביות והקשריות בהתבסס על היסטוריית השיחה

### 3. **זיכרון לטווח ארוך**
   - שומר שיחות חשובות בזיכרון לטווח ארוך
   - שולף זיכרונות רלוונטיים מבסיס הידע ומסשנים קודמים כדי להעשיר אינטראקציות חדשות
   - בונה בסיס ידע מתפתח שמשתפר עם הזמן

### 4. **שליפה חכמה של זיכרון**
   - משתמש בחיפוש סמנטי מבוסס גרף כדי למצוא מידע רלוונטי בכל הידע המאוחסן
   - מסנן חיפושים לפי תתי-קבוצות נתונים (מידע על מפתחים לעומת עקרונות)
   - משלב מקורות נתונים מרובים כדי לספק תשובות מקיפות


## 📋 דרישות מוקדמות והגדרות

### דרישות מערכת

לפני שמתחילים, ודאו שיש לכם:

1. **סביבת פייתון**
   - פייתון 3.9 או גרסה מתקדמת יותר
   - סביבת עבודה וירטואלית (מומלץ)
   
2. **מטמון Redis** (נדרש לניהול סשנים)
   - Redis מקומי: `docker run -d -p 6379:6379 redis`
   - או השתמשו בשירות Redis מנוהל
   
3. **גישה ל-LLM API**
   - מפתח API של OpenAI או ספקים אחרים (ראו [תיעוד](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **הגדרת מסד נתונים**
   - אין צורך בהגדרה כברירת מחדל. Cognee משתמשת במסדי נתונים מבוססי קבצים (LanceDB ו-Kuzu)
   - אופציונלי, ניתן להגדיר את Azure AI Search כמאגר וקטורים (ראו [תיעוד](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### הגדרת סביבה

צרו קובץ `.env` בתיקיית הפרויקט שלכם עם המשתנים הבאים:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ הבנת ארכיטקטורת הזיכרון של Cognee

### איך Cognee פועל

Cognee מספקת מערכת זיכרון מתקדמת שעולה על אחסון פשוט של מפתח-ערך:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### רכיבים מרכזיים:

1. **גרף ידע**: מאחסן ישויות, קשרים וחיבורים סמנטיים
2. **הטמעות וקטוריות**: מאפשרות חיפוש סמנטי על כל המידע המאוחסן
3. **מטמון שיחה**: שומר על הקשר השיחה בתוך ובין סשנים
4. **NodeSets**: מארגנים נתונים לקטגוריות לוגיות לצורך שליפה ממוקדת

### סוגי זיכרון במדריך זה:

- **זיכרון מתמשך**: אחסון לטווח ארוך בגרף הידע
- **זיכרון סשן**: הקשר שיחה זמני במטמון Redis
- **זיכרון סמנטי**: חיפוש מבוסס דמיון וקטורי על פני כל הנתונים


## 📦 התקנת חבילות נדרשות

התקן את Cognee עם תמיכה ב-Redis לניהול סשנים:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 אתחול סביבה וטעינת ספריות

וודאו:
1. ש-Redis פועל (לדוגמה, באמצעות Docker: `docker run -d -p 6379:6379 redis`)
2. שמשתני הסביבה מוגדרים לפני ייבוא מודולי המטמון
3. אם יש צורך, אתחלו את הליבה והריצו את התאים לפי הסדר

התא הבא יבצע:
1. טעינת משתני סביבה מתוך `.env`
2. הגדרת Cognee עם הגדרות ה-LLM שלכם
3. הפעלת מטמון לניהול סשנים
4. אימות שכל הרכיבים מחוברים כראוי


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 הגדרת תיקיות אחסון

Cognee משתמשת בשתי תיקיות נפרדות לפעולותיו:
- **Data Root**: מאחסן מסמכים שנקלטו ונתונים מעובדים
- **System Root**: מכיל את בסיס הנתונים של גרף הידע ומטא-נתונים של המערכת

ניצור תיקיות מבודדות עבור מדריך זה כפי שמפורט:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 איפוס מצב הזיכרון

לפני שנתחיל לבנות את מערכת הזיכרון שלנו, בואו נוודא שאנחנו מתחילים מחדש.

> 💡 **טיפ**: ניתן לדלג על שלב זה אם ברצונכם לשמור על זיכרונות קיימים מהריצות הקודמות שלכם כאשר תשתמשו במחברת זו מאוחר יותר.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 חלק 1: בניית בסיס הידע

### מקורות מידע עבור עוזר המפתחים שלנו

נייבא שלושה סוגי נתונים כדי ליצור בסיס ידע מקיף:

1. **פרופיל המפתח**: מומחיות אישית ורקע טכני  
2. **שיטות עבודה מומלצות ב-Python**: עקרונות ה-Zen של Python עם הנחיות מעשיות  
3. **שיחות היסטוריות**: סשנים קודמים של שאלות ותשובות בין מפתחים לעוזרי AI  

מגוון הנתונים הזה מאפשר לסוכן שלנו:  
- להבין את ההקשר הטכני של המשתמש  
- ליישם שיטות עבודה מומלצות בהמלצות  
- ללמוד מאינטראקציות מוצלחות קודמות  


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 עיבוד נתונים לגרף ידע

עכשיו נהפוך את הטקסט הגולמי שלנו לזיכרון מובנה. תהליך זה:

1. **מוסיף נתונים ל-NodeSets**: מארגן מידע לקטגוריות לוגיות
   - `developer_data`: פרופיל מפתח ושיחות
   - `principles_data`: עקרונות והנחיות לשימוש נכון ב-Python

2. **מריץ את צינור Cognify**: מחלץ ישויות, קשרים ויוצר הטמעות
   - מזהה מושגים מרכזיים
   - יוצר קשרים סמנטיים בין מידע קשור
   - מייצר הטמעות וקטוריות

זה עשוי לקחת כמה רגעים בזמן שה-LLM מעבד את הטקסט ובונה את מבנה הגרף:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 ויזואליזציה של גרף הידע

בואו נחקור את המבנה של גרף הידע שלנו. הוויזואליזציה מציגה:
- **צמתים**: ישויות שהופקו מהטקסט (מושגים, טכנולוגיות, אנשים)
- **קצוות**: קשרים וחיבורים בין ישויות
- **אשכולות**: מושגים קשורים שמקובצים לפי דמיון סמנטי

פתחו את קובץ ה-HTML שנוצר בדפדפן שלכם כדי לחקור את הגרף בצורה אינטראקטיבית:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 להעשיר זיכרון עם Memify

הפונקציה `memify()` מנתחת את גרף הידע ומייצרת חוקים חכמים על בסיס הנתונים. תהליך זה:
- מזהה דפוסים ושיטות עבודה מומלצות
- יוצר הנחיות מעשיות על בסיס התוכן
- מגדיר קשרים בין תחומי ידע שונים

חוקים אלו מסייעים לסוכן לקבל החלטות מושכלות יותר בעת מענה על שאלות. לכידת ויזואליזציה שנייה מאפשרת לך להשוות כיצד הגרף מתעבה לאחר ההעשרה.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 חלק 2: שליפה חכמה של זיכרון

### הדגמה 1: שילוב ידע בין מסמכים

עכשיו, כשהגרף הידע שלנו בנוי, נבדוק איך Cognee משלב מידע ממקורות שונים כדי לענות על שאלות מורכבות.

השאילתה הראשונה מדגימה:
- **הבנה סמנטית**: מציאת מושגים רלוונטיים גם אם לא הוזכרו במפורש
- **הצלבה בין מקורות**: שילוב פרופיל מפתח עם עקרונות Python
- **הסקה בהקשר**: יישום שיטות עבודה מומלצות על מימושים ספציפיים

### הדגמה 2: חיפוש מסונן עם NodeSets

השאילתה השנייה מראה כיצד למקד תת-קבוצות ספציפיות בגרף הידע:
- שימוש בפרמטר `node_name` לחיפוש רק בתוך `principles_data`
- מתן תשובות ממוקדות מתחום ידע מסוים
- שימושי כאשר נדרשת אינפורמציה ייחודית לתחום מסוים


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 חלק 3: הגדרת ניהול סשן

### הפעלת זיכרון שיחה

ניהול סשן הוא קריטי לשמירה על הקשר בין אינטראקציות מרובות. כאן נבצע:

1. **אתחול הקשר משתמש**: יצירה או שליפה של פרופיל משתמש למעקב סשן  
2. **הגדרת מנוע מטמון**: חיבור ל-Redis לאחסון היסטוריית השיחות  
3. **הפעלת משתני סשן**: הגדרת משתני הקשר שנשמרים בין שאילתות  

> ⚠️ **חשוב**: נדרש ש-Redis יהיה פעיל ושהערך `CACHING=true` יוגדר בסביבת העבודה שלך


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ פונקציית עזר: צפייה בהיסטוריית השיחה

פונקציה זו מאפשרת לנו לבדוק את היסטוריית השיחה המאוחסנת ב-Redis. היא שימושית עבור:
- ניפוי שגיאות בניהול סשנים
- אימות שהשיחות נשמרות במטמון
- הבנת ההקשר הזמין לסוכן


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## מפגש 1: מעבדה לתמיכה באסינכרוניות — שאלה ראשונה

התחילו את מפגש ה-`async-support-lab` על ידי בקשה לדפוסי asyncio ידידותיים לטלמטריה עבור סורק רשת מאסיבי. הגרף כבר מכיר את asyncio, aiohttp ואת שיטות המעקב, ולכן התשובה צריכה לשקף שיחות קודמות תוך התאמה לשאלה החדשה.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## בדיקת זיכרון של Session 1 לאחר ההחלפה הראשונה

הרצת `show_history(session_1)` מיד לאחר השאלה הראשונית מאשרת ש-Cognee כתבה גם את השאלה וגם את התשובה ל-Redis. אתם אמורים לראות רשומה אחת עם ההנחיה על התנהלות במקביל.


In [ ]:
await show_history(session_1)

## מפגש 1: מעקב אחר מודלי נתונים

כעת נשאל, "מתי כדאי לבחור ב-dataclasses לעומת Pydantic?" תוך שימוש באותו מזהה מפגש. Cognee אמורה לשלב את עקרונות ה-Python יחד עם שיחות FastAPI קודמות כדי לספק עצה מעמיקה—ולהראות שההקשר נשמר בתוך מפגש עם שם.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## ודא שהיסטוריית סשן 1 מכילה את שני הסבבים

קריאה נוספת ל-`show_history(session_1)` אמורה להציג שני ערכי שאלות ותשובות. זה תואם את שלב "שחזור הזיכרון" במעבדת Mem0 ומוכיח שסבבים נוספים ממשיכים את אותו תמלול.


In [ ]:
await show_history(session_1)

## מושב 2: שרשור סקירת עיצוב — מושב חדש

כדי להראות בידוד בין השרשורים, אנו מפעילים `design-review-session` ומבקשים הנחיות רישום עבור סקירות תקריות. למרות שמאגר הידע הבסיסי זהה, מזהה המושב החדש שומר על תמלילים נפרדים.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## סקירת מפגש 2 היסטוריה

`show_history(session_2)` צריך להציג רק את זוג השאלה-תשובה של סקירת העיצוב. השווה זאת עם מפגש 1 כדי להדגיש כיצד Cognee שומרת תמלילים נפרדים תוך שימוש חוזר בגרף הידע המשותף.


In [ ]:
await show_history(session_2)

## סיכום

ברכות! הרגע הענקת לעוזר הקוד שלך שכבת זיכרון ארוכת טווח המופעלת על ידי Cognee.

במדריך הזה לקחת תוכן גולמי של מפתחים (קוד, מסמכים, שיחות) והפכת אותו לגרף + זיכרון וקטורי שהסוכן שלך יכול לחפש בו, להסיק ממנו ולשפר אותו באופן מתמשך.

מה שלמדת

1. **מטקסט גולמי לזיכרון AI**: איך Cognee קולט נתונים לא מובנים והופך אותם לזיכרון חכם וחיפוש באמצעות ארכיטקטורה משולבת של וקטור + גרף ידע.

2. **העשרת גרף עם memify**: איך לעבור מעבר ליצירת גרף בסיסית ולהשתמש ב-memify כדי להוסיף עובדות נגזרות וקשרים עשירים לגרף הקיים שלך.

3. **אסטרטגיות חיפוש מרובות**: איך לבצע שאילתות בזיכרון עם סוגי חיפוש שונים (שאלות ותשובות מודעות לגרף, השלמה בסגנון RAG, תובנות, קטעים גולמיים, חיפוש קוד וכו') בהתאם לצרכי הסוכן שלך.

4. **חקירה חזותית**: איך לבדוק ולבצע דיבוג למה ש-Cognee בנה באמצעות ויזואליזציות גרף וממשק Cognee, כך שתוכל לראות בפועל איך הידע מובנה.

5. **זיכרון מודע למפגשים**: איך לשלב הקשר של מפגש עם זיכרון סמנטי מתמשך כך שהסוכנים יוכלו לזכור בין הפעלות מבלי לדלוף מידע בין משתמשים.


## עיקרי הדברים
1. זיכרון כגרף ידע המבוסס על Embeddings

    - **הבנה מובנית**: Cognee משלבת מאגר וקטורים ומאגר גרפים כך שהנתונים שלך יהיו גם ניתנים לחיפוש לפי משמעות וגם מחוברים לפי קשרים. כברירת מחדל, Cognee משתמשת במסדי נתונים מבוססי קבצים (LanceDB עבור מאגר וקטורים, Kuzu עבור מאגר גרפים).

    - **שליפה מודעת לקשרים**: תשובות יכולות להתבסס לא רק על "טקסט דומה", אלא גם על האופן שבו ישויות קשורות זו לזו.

    - **זיכרון חי**: שכבת הזיכרון מתפתחת, גדלה ונשארת ניתנת לשאילתה כגרף מחובר אחד.

2. מצבי חיפוש והסקה
    - **שליפה היברידית**: החיפוש משלב דמיון וקטורי, מבנה גרפי והסקת LLM, החל מחיפוש גולמי ועד למענה על שאלות מודע לגרף.

    - **התאמת המצב למשימה**: השתמש במצבי השלמה כאשר אתה רוצה תשובות בשפה טבעית, ובמצבי קטעים/סיכום/גרף כאשר הסוכן שלך זקוק להקשר גולמי או להניע את ההסקה שלו.

3. סוכנים מותאמים אישית ומודעים להקשר
    - **הקשר שיחה + זיכרון לטווח ארוך**: Cognee שומרת על הקשר "שיחה" קצר טווח בנפרד מזיכרון ארוך טווח ברמת משתמש או ארגון.

## יישומים בעולם האמיתי

1. **סוכני AI אנכיים**

    השתמש בתבנית מהמחברת הזו כדי להפעיל עוזרים חכמים לתחום מסוים שמבוססים על Cognee כליבת השליפה וההסקה שלהם:

- **עוזרי מפתחים**: סקירת קוד, ניתוח תקלות ועוזרי ארכיטקטורה שעוברים על קוד, APIs, מסמכי עיצוב וכרטיסים כגרף זיכרון אחד.

- **עוזרים ללקוחות**: סוכני תמיכה או הצלחה שמושכים ממסמכי מוצר, שאלות נפוצות, הערות CRM וכרטיסים קודמים עם שליפה מודעת לגרף ותשובות מצוטטות.

- **עוזרים מומחים פנימיים**: עוזרי מדיניות, משפט או אבטחה שמסיקים מתוך חוקים, הנחיות והחלטות היסטוריות מחוברות במקום מסמכי PDF מבודדים.

    Cognee ממוצבת במפורש כזיכרון מתמשך ומדויק עבור סוכני AI, ומספקת גרף ידע חי שמחליף שילובים מאולתרים של מאגרי וקטורים וקוד גרפים מותאם אישית.

2. **איחוד מאגרי נתונים לזיכרון אחד**

    אותה גישה גם עוזרת לך לבנות שכבת זיכרון מאוחדת ממקורות מפוזרים:

- **ממאגרי נתונים לגרף אחד**: קלוט נתונים מובנים (למשל, מסדי נתונים) ובלתי מובנים (למשל, מסמכים, שיחות) לגרף אחד המבוסס על Embeddings, במקום אינדקסים נפרדים לכל מערכת.

- **הסקה בין מקורות עם ציטוטים**: בצע הסקה מרובת שלבים על הכל—"חבר" לוגים, מדדים ומסמכים דרך הגרף—ועדיין החזר תשובות מבוססות עם מקור.

- **מרכזי ידע**: עבור תחומים כמו בנקאות או חינוך, Cognee כבר משמשת לאיחוד קובצי PDF, מערכות פנימיות ונתוני אפליקציות לגרף ידע אחד עם וקטורים, כך שסוכנים יכולים לענות על שאלות עם הקשר מדויק ומצוטט.

## הצעדים הבאים

יישמת את לולאת הזיכרון הבסיסית. הנה הרחבות טבעיות שתוכל לנסות בעצמך (ראה [תיעוד Cognee](https://docs.cognee.ai/) לפרטים):

1. **התנסה במודעות לזמן**: הפעל Temporal Cognify כדי לחלץ אירועים וחותמות זמן מטקסט.

2. **הכנס הסקה מונחית אונטולוגיה**: הגדר אונטולוגיית OWL עבור התחום שלך. השתמש בתמיכת האונטולוגיה של Cognee כך שהישויות והקשרים המופקים יתבססו על הסכמה הזו, מה שמשפר את איכות הגרף והתשובות הספציפיות לתחום.

3. **הוסף לולאת משוב**: תן ל-Cognee להתאים את משקלי הקשרים בגרף על סמך משוב משתמשים אמיתי, כך שהשליפה תשתפר עם הזמן במקום להישאר סטטית.

4. **כוונן להתאמה אישית והתנהגות שיחה**: השתמש במזהי משתמשים, דיירים ומאגרי נתונים כדי לתת לכל אדם או צוות תצוגה משלהם על מנוע הזיכרון המשותף.

5. **הרחב לסוכנים מורכבים יותר**: חבר את Cognee למסגרות סוכנים כדי לבנות מערכות מרובות סוכנים שכולן חולקות את אותה שכבת זיכרון. *תוסף Microsoft Agent Framework x Cognee יגיע בקרוב.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום AI [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. למידע קריטי, מומלץ להשתמש בתרגום מקצועי אנושי. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
